# 系列モデリングとディープラーニング

本章では、時系列解析で用いられるディープラーニングであるリカレントニューラルネットワーク (Recurrent Neutral Network ; 以下 RNN) について学びます。 RNN の概要と基本的な構造を解説し、RNN から派生した Long-Short Term Memory (以下 LSTM) 、Gated Recurrent Unit (以下 GRU) といったアルゴリズム、時系列モデルなどの精度向上に用いられる Attention の概要についても解説します。

## 本章の構成

- リカレントニューラルネットワーク (RNN)
- Long-Short Term Memory (LSTM)
- Attention

## リカレントニューラルネットワーク (RNN)

本章までに取り扱ってきたニューラルネットワークは流れが一方向（入力から出力まで）のフィードフォワードと呼ばれるタイプのネットワークでした。このフィードフォワードのネットワークでは、1 サンプル毎が独立し、サンプル間に関連性のない表データや画像データに対しては活用できる事を学びました。しかし、時系列データのようなサンプル間の前後関係を考慮するようなタイプのデータに対してはうまく学習を行う事ができません。  

RNN はこういった時系列を考慮して学習することを可能にするニューラルネットワークのモデルになります。これまでのフィードフォワードタイプのニューラルネットワークと RNN の違い、RNN の中身について理解していきます。  

### フィードフォワードタイプのニューラルネットワークと RNN の違い

まずはこれまでの取り扱ってきたニューラルネットワークの順伝播の流れを確認しましょう。  

![フィードフォワードの順伝播](http://drive.google.com/uc?export=view&id=1Tuca3aVGvAAOb15GBuqeSTI2gS8_SFA7)

今回取扱うデータは時系列データを想定します。工場のセンサーデータや、ウェブサイトのログデータ、株価などのデータが例としてあげられます。これらのデータの共通点はそれぞれのデータが前後関係を持ち、データ間に繋がりがあることです。  

これまで取り扱って来たデータはサンプル毎（表データの 1 行毎や画像毎）にそれぞれが独立していました。そのため上記のようにサンプル 1 とサンプル 2 の順伝播を行う際にはその関係性を考慮する必要がありませんでした。  

しかし、時系列データの場合はこのサンプル間の関係性も重要となります。RNN ではその前後関係を考慮できるようなモデルとなっています。下記の図を確認し、RNN の計算のイメージを掴みましょう。中間層が RNN 層になります。    

![RNN の順伝播](http://drive.google.com/uc?export=view&id=1GQev1HZP_03QDgw7IZtWi21108z0cFif)

図内では $h_1$ と $h_2$ という ２ つの新しい文字があります。これらの値は単純にサンプル 1 に対する中間層のノードの値になります。この値を**状態 (states）** とも呼びます。RNN では、このように 1 サンプル前の状態を引き継ぐことによってサンプル間の繋がりを表現するモデルとなっています。  

全体感が理解できたところで、次に RNN に関する文献を読む場合や実装する場合に必要となる概念を抑えておきましょう。  

- **時刻 (time steps)** : 図内のサンプル 1 が時刻 1 、サンプル 2 が時刻 2 と表現することができます。表現にはしばしば $t$ が用いられます。$\bf x_t$ が表すのは時刻 $t$ の入力変数になります。  
- **隠れ状態ベクトル (Hidden states)** : 状態を表す $\bf h$ を指します。

### RNN の計算

RNN がどのようにフィードフォワードタイプのニューラルネットワークと異なるのかを理解しました。次に具体的にどのように RNN が状態を引き継ぐのかについて簡単に確認します。  

![RNN の計算 1](http://drive.google.com/uc?export=view&id=1BgDpPm5e5vPbz2_BKdc1te-IcWD58Ikf)

このように隠れ状態ベクトル $\bf h$ はバイアスの用に中間層の出力に追加され、次の計算に使用されます。もし次の層が全結合層の場合これまでと同様の計算方法になります。  


#### tanh 関数

RNN や後述の LSTM などでは活性化関数にハイパボリック・タンジェント関数 (hyperbolic tangent) が一般的に用いられます。このハイパボリック・タンジェント関数は tanh 関数とも呼ばれます。（本資料では tanh 関数と呼びます。）  

数式は下記になります。  

![tanh 関数](https://wikimedia.org/api/rest_v1/media/math/render/svg/84c428bf21e34ccc0be8becf3443b06a4b61f3ee)

![tanh 関数 図](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Activation_tanh.svg/240px-Activation_tanh.svg.png)

*出典 : Wikipedia*  
　
 
tanh 関数の出力は -1 ~ 1 の間の値を取ります。これまでは ReLU 関数を用いる事が一般的と伝えましたが、ReLU 関数の特性として、マイナスの値は全て 0 にする事がありました。これでは状態を引き継ぐ特性を持つ RNN の情報がうまく引き継ぐことができない可能性があります。そのため出力を -1 ~ 1 の間におさめ、状態（出力）の強弱を表現する事が可能な tanh 関数を用いる事が一般的となっています。  


#### RNN の問題点

実際 RNN はあまり使用される事は昨今では少なくなり、RNN の派生モデルである LSTM や GRU といったモデルが用いられるケースが多いです。その理由は RNN には下記に挙げる 2 つの問題点があるためです。  

- 長期的なデータの関係性を学習することがうまくいかない
- 勾配消失もしくは勾配爆発が起こる

1 つ目の問題点である長期的なデータの関係性を学習することがうまくいかないという問題点について考えます。RNN の計算の流れを思い出してください。RNN の計算は単純な行列の積と和を計算し、tanh 関数を適用するといった構成からなっていました。そして、状態の引き継ぎは 1 つ前の中間層の出力を使用しました。  
こういった計算方法では時刻 $t$ の長さが長くなればなるほど、うまく状態を引き継ぐ事ができない問題が生じます。  

2 つ目の問題点である勾配消失もしくは勾配爆発は、活性化関数に使用している tanh 関数が関係しています。これまで使用した ReLU 関数は入力が　 0 より大きければ、出力はその値をそのまま使用していました。これにより勾配の劣化（小さくなる事）を避け、勾配がなくなってしまうという問題をうまく回避できていました。しかし、RNN では基本的に tanh 関数を活性化関数として用いるため、勾配消失の問題が起こりやすくなっています。詳細に関しては[こちら](https://en.wikipedia.org/wiki/Vanishing_gradient_problem)を参照して下さい。  

## Long-Short Term Memory (LSTM)

前節の最後では RNN にはどのような問題が考えられるかを確認しました。本節では時系列データのモデリングによく用いられる LSTM について確認し、どのような機構を持ち、RNN の問題に対処しているかを理解します。  

### 概要

LSTM も基本的な計算の考え方は RNN と同じになります。異なる点は LSTM が 4 つの新たな機構を持つ点になります。ここでは、それぞれの計算方法について確認します。  

まずはじめに大きく RNN と LSTM が異なる点について確認します。LSTM には**記憶セル (memory cell)** と呼ばれる LSTM 専用の記憶部のようなものが存在します。   

この記憶セルにはその名前の通り、記憶のような過去の情報を保つ役割があります。そして、この記憶セルは隠れ状態ベクトルのように出力される事はなく、LSTM 内のみで保持されます。  

![記憶セル](http://drive.google.com/uc?export=view&id=1rPDQFgPuQcYhlNA0gEAMEDKfya8gQf9o)

また、今後は上記のような簡略化した図で LSTM を表現します。LSTM 層からは引き継ぐ状態 ($\bf c_t$ と $\bf h_t$) と次の層（図内では出力層）へと $\bf h_t$ が出力されます。  
※上図では $\bf c_t$ と $\bf h_t$ をそれぞれ別途記載していますが、以降は 1 つの線で統一して表現します。  


図内の LSTM 層（ノード）内の中身について確認していきます。中身の計算の流れは下記のようになっています。  
※LSTM 内の計算方法は 1 つではなく、様々な型が存在します。その中でも最もシンプルかつ代表的なものである忘却ゲートを持つ LSTM を紹介していきます。  

![LSTM 計算グラフ](http://drive.google.com/uc?export=view&id=138M7-z6p8vwlDAohowJqQDaPnPiEaeMA)

先程の RNN と比較すると少し複雑な中身になっていることが確認できます。  
まず着目するポイントは下記の 4 つになります。   

- Output ゲート ($\bf o$) : 出力の割合の決定
- Forget ゲート ($\bf f$) : 何を忘れるかを決定
- 新しい記憶セル ($\bf c$) : 新しく覚えるべき情報の追加
- input ゲート ($\bf i$) : 追加情報にどれだけ価値があるかを判断


#### sigmoid 関数と tanh 関数

4 つのポイントの詳細を確認する前に、sigmoid 関数と tanh 関数の役割について理解する必要があります。  

![sigmoid 関数と tanh 関数](http://drive.google.com/uc?export=view&id=1lWY8dXlXVVfTgOKNHlWlWSrB-JKo-eAv)

それぞれの関数の LSTM 内での役割をまとめると下記になります。   

| 関数    | 出力の範囲 | 役割                                       |
| ------- | ---------- | ------------------------------------------ |
| sigmoid | 0 ~ 1      | 入ってきた情報を次にどれだけ流すのかの割合       |
| tanh    | -1 ~ 1     | 入ってきた情報の強弱（度合い）を表現 |

### LSTM の計算

それではそれぞれの計算の中身を式で確認し、それぞれの役割の理解を深めましょう。  


#### forget ゲート

*何を忘れるかを決定*  

![forget ゲート](http://drive.google.com/uc?export=view&id=1lLoxfXj1D6_DISF0InB2OhShlDH6n72y)

$$
{\bf f}_t = \sigma({\bf x}_t W_x^{(f)} + h_{t-1} W_h^{(f)} + {\bf b}^{(f)})
$$

LSTM の計算ではゲートごとにパラメータが存在します。一見複雑に見える計算も中身は基本的に線形変換と sigmoid 関数（もしくは tanh 関数）の適用になります。
forget ゲートの役割を理解するためには後述の新しい記憶セルの部分の計算を理解する必要があります。  


#### input ゲート

*追加情報にどれだけ価値があるかを判断*

![input ゲート](http://drive.google.com/uc?export=view&id=1Ai3A6JpP5amPCwHR3Y3UE8zpfc4PGnpK)

$$
{\bf i}_t = \sigma({\bf x}_tW_x^{(i)} + h_{t-1} W_h^{(i)} + {\bf b}^{(i)})
$$

パラメータは異なりますが、計算内容は前述の forget ゲートと同様になります。  
input ゲートも forget ゲートと同様に役割を理解するためには次の新しい記憶セルの部分の計算を理解する必要があります。  


#### 新しい記憶セル

*新しく覚えるべき情報の追加*  

![新しい記憶セル](http://drive.google.com/uc?export=view&id=1O3iVgcR0rGR17kB6jX5j9q_OFq4RF5v9)

$$
{\bf g}_t = \tanh({\bf x}_tW_x^{(g)} + h_{t-1} W_h^{(g)} + {\bf b}^{(g)})
$$

$$
{\bf c}_t =　{\bf f}_t \odot {\bf c}_{t-1} + {\bf i}_t \odot {\bf g}_t
$$

$\odot$ は[要素積](https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%80%E3%83%9E%E3%83%BC%E3%83%AB%E7%A9%8D)といい、行列内の成分ごとの積による定まる行列の積になります。（要素積はアダマール積とも呼ばれます。）  

では、forget 、input ゲートの役割も含め確認していきます。  
2 つ目の式を確認して下さい。第 1 項の  

$$
{\bf f}_t \odot {\bf c}_{t-1}
$$　

は、前時刻の記憶セルベクトル (${\bf c}_{t-1}$) と forget ゲートの出力 (${\bf f}_t$) の要素積となっています。もし、forget ゲートの出力の値が全て 0 だった場合を想定して下さい。その場合、${\bf c}_{t-1}$ の値は全て 0 となり、保持していた記憶セルの情報が全て失われます。そのため forget ゲートのパラメータはどの記憶セルの情報を保持し、忘れるのかを決定するように学習されます。  

続いて第 2 項を確認します。  

$$
{\bf i}_t \odot {\bf g}_t
$$

input ゲートの出力の値 (${\bf i}_t$) と ${\bf g}_t$ の要素積となっています。${\bf g}_t$ の計算に用いられている活性関数は tanh 関数です。そのため、出力は -1 ~ 1 の間の値を取ります。そのため、${\bf g}_t$ は input ゲートの出力をどれだけ反映するかの度合いの調整をしていると言えます。

上記第 1 項、第 2 項の和が ${\bf c}_t$ となります。どの情報を保持するのか調整された値と、どの情報を新たに取得するのか調整された値の 2 つが合わさったものが次の時刻の記憶セル、そして出力値（隠れ状態ベクトル）に使用されます。  


#### output ゲート

![output ゲート](http://drive.google.com/uc?export=view&id=1rs3ExaykvsQRz1E1nxU68ntFJv3x3EEY)

*出力の割合の決定*  

$$
{\bf o}_t = \sigma({\bf x}_tW_x^{(o)} + h_{t-1} W_h^{(o)} + {\bf b}^{(o)})
$$

この計算は RNN で確認したものと同様になります。単純に 1 時刻前の出力と現時刻の入力を用いて線形変換を行っています。そして、sigmoid 関数を通して最終的な出力の割合を決定しています。  


#### 隠れ状態ベクトル

![隠れ状態ベクトル](http://drive.google.com/uc?export=view&id=1n9wIaVvKAtBVhNIGoreVZwQ0qz-i8WQy)

$$
{\bf h}_t = {\bf o}_t\odot\tanh{\bf c}_t
$$

新しい記憶セルに tanh 関数を適用し、記憶セル内の情報の度合いを調整し、どれだけ出力するのかの調整を行っていると言えます。  

このようにして、記憶セルと呼ばれる新たな機構を用いる事により、LSTM は RNN の課題であった長期的状態の保存を可能にしていると言われています。  

Gated Recurrent Units (以下 GRU) は LSTM 同様の RNN から派生したモデルの 1 つになります。違いとしては記憶セルを保持しない事にあります。そのため GRU は LSTM の簡易版と呼ぶことができます。記憶セルを持たないことからより効率的（時間を短縮して）に学習が可能と言われています。しかし、どちらのモデルがいいという確実な答えは無いため、実際に実装する際にはどちらのモデルも試し、予測精度の高いものを選択するといいでしょう。    

## Attention

LSTM は状態を保持でき、データ間の前後関係を把握することが理解できました。LSTM は時系列データを用いて、未来の予測を行う問題設定で用いられるだけでなく、よく自然言語の領域で用いられます。LSTM の前後関係を理解する事ができるということは文章の**文脈 (context)** を学習することができるということだからです。2020 年初めの現在では LSTM などのモデルは未来予測よりも自然言語の領域で用いられる事が多いのが実情です。   

LSTM がどのように自然言語の分野で活躍するのかを確認します。そして、そこからより発展的な技術である Attention について解説します。  


### 機械翻訳のアルゴリズム

本節では英日翻訳の問題設定に解説を進めます。LSTM を用い機械翻訳を行代表的なアルゴリズムの 1 つである Seq2Seq について確認しましょう。  

下記の図は Seq2Seq のアルゴリズムになります。  

![Seq2Seq 推論](http://drive.google.com/uc?export=view&id=1IcavuUqxiw0lALjcdCwyczMJdVnjqbq-)

※図内 LSTM 層のループは時刻 $t$ までを引き継ぐ役割を表しています。

Seq2Seq のモデルには大きく Encoder と Decoder の 2 つの機構が存在します。今回の英日翻訳を例に解説します。（詳細に関しては第 10 章で解説します。）  

- Encoder : 英文の情報を取得し、Decoder に引き継ぐ。LSTM 層の状態を引き継ぐ特性により文章の前後関係を汲み取る。
- Decoder : Encoder から引き継いだ状態を用いて、翻訳を行う。Decoder の初めの入力値には BoS のような文の始まりを表すものを取る。Dense 層（全結合層）では単純に LSTM 層からの入力を用いて、辞書内の日本語の分類を行う。

ここでは、Encoder で入力の英文の状態を取得し、Decoder でその状態と予測値を用いて翻訳を行う点を抑えておいて下さい。  


#### LSTM の問題点

一見 LSTM を用いることにより、文脈の情報を引き継ぎ翻訳がうまくいくことができそうに感じます。  
しかし、Encoder の出力は最終的に $\bf h_t$ と $\bf c_t$ の 2 つのベクトルにまとめられてしまいます。この情報量ではエンコードを行う文章が長くなった場合や複雑な場合にうまく情報を Decoder に伝える事ができません。  

このような問題の解決策として挙げられるのが Attention と呼ばれる手法になります。  

※Attention についての明確な定義は存在しません。一般的には入力されたデータのうち、どの部分を重視するかを決定する手法を指します。本資料では自然言語の領域に絞って解説しますが、Attention は画像や音声認識など幅広い領域で使用される手法になります。


### Attention の概要

Attention は前述の LSTM の問題点に対して、Encoder の入力を Decoder 側から直接参照できるようにします。参照する際にどの単語に対して、注意 (Attention) を払うべきなのかまたは、どの単語を無視するべきなのかの決定を行います。  

※本資料では　Seq2Seq に Attention を用いる場合の手法について絞って解説します。  

例えば下記のような文章の〇〇の部分を予測したいとしましょう。  

「今日は天気がいい、こんな日には〇〇」  

おそらく次に来る言葉としては「外出したい」、「ピクニックに行きたい」等が候補として考えられるでしょう。こういった場合重要になるのは「こんな日には」ではなく、「天気がいい」であることがわかります。我々はこのように文章を考えるときや予測を行う際に、ある単語を重要度を上げ考えています。そして、ある単語の重要度は下げています。  

このように Attention ではどのような単語が重要であり、重要でないかの情報を決定します。  
※上記の例はイメージをつ掴みやすくする事を目的とするため、単語の区切り方や、Attention についての厳密性は完全ではありません。

#### Attention の計算の流れ

ここではこちらの論文 [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/pdf/1508.04025.pdf) から説明を行います。Attention には大きく global と local と2 種類が存在します。今回は global Attention に絞って解説を行います。  

![Attention](http://drive.google.com/uc?export=view&id=1SGmnQ3qVYTE0SUr7AbYQltzdrxXSrbsq)

*引用 : [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/pdf/1508.04025.pdf)*

上図は Attention の仕組みを表しています。横方向の四角形の繋がりは時刻ごとの LSTM 層を表しています。横方向の矢印は隠れ状態ベクトルの時刻ごとの引き継ぎを、縦方向の矢印は時刻ごとの出力（隠れ状態ベクトル）を表現しています。そして、青色の四角形が Encoder 、赤色が Decoder を表しています。    

ポイントは図内の `Attention Layer` の中にある下記 2 つになります。  

- Global align weights (${\bf \alpha}_t$)
- Context vector (${\bf c}_t$)

それぞれの計算を確認しましょう。  

$$
{\begin{align}
\alpha_t(s) = \frac{exp(score(h_t, \overline{h_s}))}{\sum_{s'} exp(score(h_t, \overline{h_{s'}}))}
\end{align}
}
$$

上記は Global align weights (${\bf \alpha}_t$) の算出を行う計算式になります。それぞれ要素ごとに分解して確認します。  

まずは計算に使用される 2 つのベクトルについて確認します。  

- ${\bf h}_t$ : Decoder の出力（隠れ状態ベクトル）
- $\overline{{\bf h}_s}$ : Encoder の**各時刻**の出力（隠れ状態ベクトル）

次に単語の重要度の算出している部分を確認します。  

$$
score({\bf h}, \overline{{\bf h}_s})
$$

$score()$ が表すのは特定の計算になります。論文内では、この計算に、単純なベクトルの内積 (${\bf h}_t^T {\bf \overline h}_s $)、重みを含んだ内積 (${\bf h}_t^T {\bf W}_a {\bf \overline h}_s $)、 結合などを提案しています。  

そして、この計算結果に対し、ソフトマックス関数を適用することでベクトルの値を 0~1 の範囲に収めます。このベクトルの値が大きいものは単語の重要度が高く、小さいものは重要度が低い事を表します。  

$$
{\begin{align}
{\bf \alpha}_t(s) = \frac{exp(score({\bf h}_t, \overline{{\bf h}_s}))}{\sum_{s'} exp(score({\bf h}_t, \overline{{\bf h}_{s'}}))}
\end{align}
}
$$

　上述の ${\bf \alpha}_t$ と Encoder の各時刻の隠れ状態ベクトルを掛け合わせ、それぞれを足し合わせます。  

$$
{{\bf c}_t=\sum_{s}{\bf \alpha}_t(s) \overline{{\bf h}_{s}}
}
$$

この Context vector (${\bf c}_t$) は入力の単語の重要度を考慮した上で、Encoder の情報を持ったベクトルであると言えます。  

この Context vector (${\bf c}_t$) と Decoder の隠れ状態ベクトル (${\bf h}_t$) を結合し、全結合層の入力とします。  

$$
{\tilde{{\bf h}_t} = tanh({\bf W}_c[{\bf c}_t;{\bf h}_t])}
$$

このようにして、単語の重要度を考慮した Encoder の情報を取得し、分類を行うことができることがわかりました。  
翻訳のディープラーニングのモデルの中には Attention のみを用いたものなども存在します。また、画像系のディープラーニングにも Attention を用いるケースも増えてきています。  

発展的な内容について理解するため、今回紹介した Attention の基礎はしっかり抑えておきましょう。  

## 練習問題 本章のまとめ

本章では、新たな知識のインプットが他の章よりも多くなっています。そのため本章の練習問題はなしとし、学んだ内容をもう一度確認し直して下さい。  

次章では、LSTM の実装を行いますが、本章の内容を把握していることが前提となっている上、内容も少しレベルが高いものとなっています。しっかり本章の内容を理解し、次の章に進んで下さい。  

---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学